Snowflake Cortex AI ofrece dos funciones específicas que permiten realizar fácilmente análisis de sentimiento en datos de texto, como comentarios o reseñas de clientes.

- `ENTITY_SENTIMENT` ofrece información general y granular sobre el sentimiento de cada entidad. Es ideal para casos de uso que requieren análisis de sentimiento para aspectos específicos del texto o una identificación matizada del sentimiento: positivo, negativo, neutral y mixto.
- `SENTIMENT` proporciona una puntuación numérica del sentimiento que identifica el grado de negatividad o positividad. Está diseñado para casos de uso que priorizan la velocidad y la rentabilidad sobre el análisis granular a nivel de entidad.

### `ENTITY_SENTIMENT`

Cortex AI `ENTITY_SENTIMENT` captura un espectro de opiniones de clientes que va más allá de las categorías generales positivas, negativas y neutrales. Para el análisis de sentimiento basado en aspectos, debemos especificar el contenido (como un comentario o una reseña de un cliente) y las entidades (aspectos o categorías) para las que deseamos analizar el sentimiento. `ENTITY_SENTIMENT` nos devolverá puntuaciones de sentimiento para cada entidad, así como una puntuación de sentimiento general.

### Ejemplo

En el siguiente ejemplo vamos a crear una tabla con reviews de un restaurante de nombre “La cubana” y vamos a utilizar la función `ENTITY_SENTIMENT` para ver el sentimiento de las personas respecto a 4 puntos claves: costo de la comida, calidad, servicio y tiempos de espera.

In [ ]:
-- Establecer el contexto
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

-- Crear tabla para las opiniones de restaurante
CREATE OR REPLACE TABLE reviews_restaurante (
   usuario VARCHAR(50),
   opinion VARCHAR(1000)
);

In [ ]:
-- Insertar 10 filas con revisiones variadas
INSERT INTO reviews_restaurante (usuario, opinion)
VALUES
   ('juan20', 'La comida en "La Cubana" es excelente, aunque un poco cara para la porción que sirven. Los platillos tradicionales cubanos son auténticos y llenos de sabor. El servicio fue atento pero la espera para recibir el plato principal fue de casi 30 minutos.'),
   
   ('maria_lopez', 'Me encantó el ambiente de "La Cubana", muy acogedor. La relación calidad-precio es justa y el servicio excepcional. La mesera conocía perfectamente el menú y nos recomendó platos deliciosos. Solo tardaron 15 minutos en servirnos.'),
   
   ('foodlover45', 'Decepcionado con "La Cubana". Esperamos más de 45 minutos para ser atendidos y los platos estaban fríos cuando llegaron. Además, los precios son demasiado elevados para lo que ofrecen. No volveré.'),
   
   ('carlos_gomez', 'El ropa vieja de "La Cubana" es el mejor que he probado en la ciudad. El servicio fue rápido y eficiente. Precios moderados considerando la calidad de los ingredientes. Definitivamente regresaré.'),
   
   ('chef_patricia', 'Como profesional de la gastronomía, puedo decir que "La Cubana" mantiene la autenticidad de los sabores caribeños. El servicio es un poco lento en horas pico, pero la calidad de la comida lo compensa. Los precios son razonables para un restaurante de especialidad.'),
   
   ('roberto87', 'Fui a "La Cubana" en mi cumpleaños y la experiencia fue increíble. El personal fue muy atento, la comida llegó en tiempo récord y la relación calidad-precio es inmejorable. Los cócteles cubanos son espectaculares.'),
   
   ('foodie_ana', 'Esperaba mucho más de "La Cubana". Los platos son mediocres y extremadamente caros para lo que ofrecen. El servicio fue correcto pero nada especial. Tardaron 25 minutos en traer una simple ensalada.'),
   
   ('david_castro', '"La Cubana" ofrece una experiencia gastronómica completa. La calidad de los ingredientes es notable, el servicio atento y personalizado. Los precios son un poco elevados, pero acordes a lo que recibes. La espera fue razonable, unos 20 minutos.'),
   
   ('elena_viajera', 'Como cubana, puedo certificar que los sabores de "La Cubana" son bastante auténticos. El servicio fue amable pero desorganizado, y los precios son justos. Tuvimos que esperar casi 40 minutos por nuestra comida en un día poco concurrido.'),
   
   ('gourmet2023', 'Balance perfecto entre calidad y precio en "La Cubana". El servicio es impecable, los camareros conocen a la perfección cada plato y sus ingredientes. La espera fue mínima y la comida vale cada centavo. El mojito es simplemente perfecto.')
;

Ahora usaremos `ENTITY_SENTIMENT` para ver el sentimiento de las personas respecto a 4 puntos mencionados.

In [ ]:
SELECT usuario, SNOWFLAKE.CORTEX.ENTITY_SENTIMENT(opinion,['costo de la comida', 'calidad', 'servicio', 'tiempos de espera']) FROM curso_ia.seccion_2.reviews_restaurante;

Veamos que sucede si proporcionamos entidades no relacionadas con el contenido de la reseña del restaurante. Para el ejemplo proporcionaremos la entidad "dormitorios" y veremos como el modelo la clasifica como "unknown"("desconocida").

In [ ]:
SELECT usuario, SNOWFLAKE.CORTEX.ENTITY_SENTIMENT(opinion,['dormitorios', 'calidad', 'servicio']) FROM curso_ia.seccion_2.reviews_restaurante;

Si solo deseamos el sentimiento general, debemos especificar el contenido sin entidades de análisis como se muestra a continuación.

In [ ]:
SELECT usuario, SNOWFLAKE.CORTEX.ENTITY_SENTIMENT(opinion)
    FROM curso_ia.seccion_2.reviews_restaurante;

### **`SENTIMENT`**

La función `SENTIMENT` de Cortex AI devuelve el sentimiento general como una puntuación entre -1.0 y 1.0. Cuanto más cercano a 1, mayor es la probabilidad de que el texto tenga un sentimiento positivo; cuanto más cercano a -1, mayor es la probabilidad de que el texto tenga un sentimiento negativo. La siguiente tabla ofrece orientación sobre cómo interpretar las puntuaciones de sentimiento:

| **Sentimiento** | **Puntuación** |
| --- | --- |
| Positive | 0.5 to 1 |
| Neutral | -0.5 to 0.5 |
| Negative | -0.5 to -1 |

Veamos un ejemplo utilizando la misma tabla de reviews que hemos creado con anterioridad y aplicando la tabla anterior para clasificar el sentimiento.

In [ ]:
SELECT
    usuario,
    SNOWFLAKE.CORTEX.SENTIMENT(opinion) AS score,
    CASE 
        WHEN score > 0.5 THEN 'Positivo'
        WHEN score < -0.5 THEN 'Negativo'
        ELSE 'Neutral'
    END AS sentimiento
FROM curso_ia.seccion_2.reviews_restaurante;

### Restricciones del modelo

Todos los modelos disponibles en Snowflake Cortex AI tienen limitaciones en el número total de tokens de entrada y salida, lo que se conoce como la ventana de contexto del modelo.

La ventana de contexto de `ENTITY_SENTIMENT` está configurada para que el modelo mantenga un alto nivel de precisión. 

`ENTITY_SENTIMENT` se entrenó y optimizó para entradas de texto de 2048 tokens (**aproximadamente 1600 palabras**), y se puede pasar **un máximo de 10 categorías de entidades**, cada una con un máximo de 30 caracteres.

| **Función** | **Ventana de contexto (tokens)** | **Max Entidades** |
| --- | --- | --- |
| ENTITY_SENTIMENT | 2,048 | 10 |
| SENTIMENT | 512 | n/a |

### `SENTIMENT` en Python

El siguiente ejemplo muestra como podemos hacer uso de la función `SENTIMENT` desde Python.

In [ ]:
# Importar las liberías
from snowflake.snowpark.context import get_active_session
from snowflake.cortex import sentiment
from snowflake.snowpark.functions import col, when, lit


# Obtener la sesión de snowpark activa
session = get_active_session()

In [ ]:
# Leemos la tabla que contiene las reseñas
reviews = session.table('curso_ia.seccion_2.reviews_restaurante')

In [ ]:
# Utilizamos la función sentiment
reviews_sentimientos = (
    reviews
        .with_column('Score', sentiment(col('opinion')))
        .with_column('Sentimiento',
                     when(col('Score') > 0.5, lit('Positivo'))
                     .when(col('Score') < -0.5, lit('Negativo'))
                     .otherwise('Neutro')
                    )
        
)

reviews_sentimientos.show()